In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as opt

# Simulating a dataset with alternative-specific variables, adding Gumbel noise for realism
np.random.seed(42)
N = 100000  # Number of individuals
J = 3    # Number of alternatives (Car, Bus, Train)

# Alternative-specific attributes (Price, Travel Time) - same across individuals
prices = np.array([15, 5, 10])  # Price for Car, Bus, Train
travel_times = np.array([30, 45, 40])  # Travel time for Car, Bus, Train

# True preference weights (Beta values)
beta_price = -0.5  # Base price sensitivity
beta_time = -0.1  # Sensitivity to travel time

# Compute utilities
U = beta_price * prices + beta_time * travel_times  # Shape (J,)
U = np.tile(U, (N, 1))  # Repeat for all individuals (N, J)

# Add Gumbel-distributed noise to utilities
epsilon = np.random.gumbel(0, .1, size=(N, J))  # Shape (N, J)
U += epsilon

# Simulate choices using softmax probabilities
def softmax(U):
    exp_U = np.exp(U - np.max(U, axis=1, keepdims=True))  # Prevent overflow
    return exp_U / exp_U.sum(axis=1, keepdims=True)

P = softmax(U)  # Choice probabilities
choices = np.array([np.random.choice(J, p=P[i]) for i in range(N)])

# Organizing data into a DataFrame
df = pd.DataFrame({
    "Individual": np.repeat(np.arange(N), J),
    "Mode": np.tile(["Car", "Bus", "Train"], N),
    "Chosen": (np.tile(np.arange(J), N) == np.repeat(choices, J)).astype(int),
    "Price": np.tile(prices, N),
    "TravelTime": np.tile(travel_times, N)
})

# Log-likelihood function for estimation
def log_likelihood(beta, X, choices):
    U = X @ beta  # Compute utilities
    P = softmax(U)  # Compute probabilities
    chosen_probs = P[np.arange(N), choices]  # Get probability of chosen option
    return -np.sum(np.log(chosen_probs))  # Negative log-likelihood

# Prepare data for estimation
X_columns = ["Price", "TravelTime"]
X = df[X_columns].values.reshape(N, J, -1)  # Reshape for estimation
choices = df["Chosen"].values.reshape(N, J).argmax(axis=1)  # Convert to choice indices

# Estimating parameters using MLE
initial_beta = np.zeros(X.shape[2])  # Start with zero coefficients
result = opt.minimize(log_likelihood, initial_beta, args=(X, choices), method='BFGS')

# Print results
print("Estimated Beta:", result.x)
print("True Beta:", [beta_price, beta_time])

Estimated Beta: [-0.49854674 -0.09891729]
True Beta: [-0.5, -0.1]
